In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

sibi_path = os.getenv("SIBI_PATH")
bisindo_path = os.getenv("BISINDO_PATH")

# ====== Folder dataset ======
raw_dirs = [sibi_path, bisindo_path]

processed_dir = "../../dataset/processed_letters"
os.makedirs(processed_dir, exist_ok=True)

mp_holistic = mp.solutions.holistic

# ====== Inisialisasi Holistic sekali ======
holistic = mp_holistic.Holistic(static_image_mode=True)

# ====== Fungsi extract enhanced features ======
def extract_enhanced_features(lh, rh):
    # lh, rh shape: (21*3,)
    def compute_features(hand):
        lm = hand.reshape(21, 3)
        features = []
        wrist = lm[0]
        tips = [4, 8, 12, 16, 20]
        # jarak wrist ke ujung jari
        for t in tips:
            features.append(np.linalg.norm(lm[t] - wrist))
        # jarak antar ujung jari
        for i in range(len(tips)):
            for j in range(i+1, len(tips)):
                features.append(np.linalg.norm(lm[tips[i]] - lm[tips[j]]))
        # sudut antar sendi
        chains = [[0,1,2,3,4],[0,5,6,7,8],[0,9,10,11,12],[0,13,14,15,16],[0,17,18,19,20]]
        for chain in chains:
            for i in range(len(chain)-2):
                v1 = lm[chain[i+1]] - lm[chain[i]]
                v2 = lm[chain[i+2]] - lm[chain[i+1]]
                cos_angle = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)+1e-8)
                features.append(np.arccos(np.clip(cos_angle,-1,1)))
        return np.array(features)
    
    lh_feat = compute_features(lh)
    rh_feat = compute_features(rh)
    return np.concatenate([lh, rh, lh_feat, rh_feat])

# ====== Fungsi ekstrak keypoints + augmentasi ======
def extract_keypoints(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = holistic.process(img_rgb)

    # Pose (opsional, bisa diikutsertakan)
    pose = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks else np.zeros(33*3)

    # Hands
    lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks else np.zeros(21*3)

    # ====== Enhanced features ======
    enhanced_features = extract_enhanced_features(lh, rh)

    # ====== Augmentasi sederhana ======
    shift = np.random.uniform(-0.02, 0.02, enhanced_features.shape)
    enhanced_features += shift

    return enhanced_features

# ====== Proses semua gambar ======
for raw_dir in raw_dirs:
    for label in os.listdir(raw_dir):
        label_path = os.path.join(raw_dir, label)
        if not os.path.isdir(label_path):
            continue

        target_label_dir = os.path.join(processed_dir, label)
        os.makedirs(target_label_dir, exist_ok=True)

        for file in os.listdir(label_path):
            if file.endswith((".jpg", ".png")):
                img_path = os.path.join(label_path, file)
                kp = extract_keypoints(img_path)
                save_path = os.path.join(target_label_dir, file.replace('.jpg','.npy').replace('.png','.npy'))
                np.save(save_path, kp)
                print(f"✅ Saved keypoints {save_path}")

holistic.close()
print("✅ Semua gambar dari SIBI + BISINDO diproses dengan enhanced features siap untuk 1D-CNN.")


I0000 00:00:1760379315.433207       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3


✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220221_172302.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220221_175647.npy
✅ Saved keypoints ../../dataset/processed_letters/R/1645716117.9940615.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220221_153904.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220221_112958.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220221_105217.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220224_174004.npy
✅ Saved keypoints ../../dataset/processed_letters/R/20201031_134139_002.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220224_174001_1.npy
✅ Saved keypoints ../../dataset/processed_letters/R/1645946950.1533844_L.npy
✅ Saved keypoints ../../dataset/processed_letters/R/20201018_121534_016.npy
✅ Saved keypoints ../../dataset/processed_letters/R/IMG_20220224_174011.npy
✅ Saved keypoints ../../dataset/processed_letters/R/1645716139.6726453.npy
✅ Saved key